In [ ]:
import os
import numpy as np
import spectral
import glob
import utils


def convert_to_reflectance(input_hdr_path, output_hdr_path, gain_path, offset_path):
    """
    Converts a raw ENVI image to reflectance using gain and offset files.

    Args:
        input_hdr_path (str): Path to the input raw ENVI header file.
        output_hdr_path (str): Path to save the output reflectance ENVI header file.
        gain_path (str): Path to the .npy file containing the gain values.
        offset_path (str): Path to the .npy file containing the offset values.
    """
    try:
        print(f"Converting '{os.path.basename(input_hdr_path)}' to reflectance...")

        # number of smoothing iterations
        smoothing_level = 2
        # wl range(s) to remove (in nanometers)
        bbl_wl_ranges = [[0,425], [750,770], [900,2500]]
        
        # 1. Load gain and offset
        gain = np.load(gain_path)
        offset = np.load(offset_path)

        # 2. Open the input image
        im = spectral.open_image(input_hdr_path)
        im.wl = np.asarray(im.bands.centers)
        wl = im.wl


        # ====== Convert to Reflectance ======
        # wl range(s) to remove (in nanometers)
        bbl_wl_ranges = [[0,425], [750,770], [900,2500]]
        indices = []
        for i in range(len(wl)):
            is_bad_band = False
            for bbl_wl_range in bbl_wl_ranges:
                if bbl_wl_range[0] < wl[i] < bbl_wl_range[1]:
                    is_bad_band = True                
            if (not is_bad_band):
                indices.append(int(i))
        indices = np.asarray(indices, dtype=np.int16)

        # determine the parameters for the image
        nr = im.nrows
        nc = im.ncols
        nb = len(indices)

        # subset the gain and offset to the good bands
        gain = gain[indices]
        offset = offset[indices]
        wl = wl[indices]

        # Prepare an output array for the reflectance image
        imRef = np.zeros((nr, nc, nb), dtype=np.float32)
        # Create the data mask
        mask = (im.read_band(0) > 0).astype(np.float32)
            
        # ====== Load the image and compute reflectance ======
        # Loop over bands and fill the result
        print('Reading the image and converting to reflectance.')
        for i, b in enumerate(indices):
            imRef[:, :, i] = (gain[i]*np.squeeze(im.read_band(b) + offset[i])*mask).astype(np.float32)
                                    
        # ====== Spatially smooth the image ======
        for i in range(smoothing_level):
            print(f'Smoothing the image, iteration {i+1}.')
            imRef = utils.spatial_smoothing(imRef, mask=mask).astype(np.float32)

        # ====== Save the image ======
        # Save the image
        print('Saving the image.')
        md=im.metadata
        md['wavelength'] = [str(w) for w in wl]

        spectral.envi.save_image(
            output_hdr_path,
            imRef,
            metadata=md,
            force=True
        )
        print(f"  -> Successfully saved to: {output_hdr_path}")
    
    except FileNotFoundError as e:
        print(f"ERROR: File not found during conversion. Details: {e}")
    except Exception as e:
        print(f"ERROR: An unexpected error occurred during conversion of {input_hdr_path}. Details: {e}")


def batch_convert_to_reflectance(input_source, output_dir, gain_path, offset_path):
    """
    Batch converts raw ENVI images to reflectance.

    Args:
        input_source (str or list): Can be one of:
                                    - A path to a single ENVI .hdr file.
                                    - A path to a directory containing ENVI .hdr files.
                                    - A list of paths to one or more ENVI .hdr files.
        output_dir (str): Path to the directory where corrected files will be saved.
        gain_path (str): Path to the .npy file containing the gain values.
        offset_path (str): Path to the .npy file containing the offset values.
    """
    # --- 1. Validate paths and create output directory ---
    os.makedirs(output_dir, exist_ok=True)
    print(f"Converted files will be saved in: '{os.path.abspath(output_dir)}'")

    # --- 2. Determine input type and get list of files ---
    envi_files = []
    if isinstance(input_source, list):
        envi_files = [f for f in input_source if os.path.isfile(f) and f.lower().endswith('.hdr')]
        invalid_files = [f for f in input_source if f not in envi_files]
        if invalid_files:
            print(f"Warning: The following paths were invalid or not .hdr files and will be skipped: {invalid_files}")
    elif isinstance(input_source, str):
        if os.path.isdir(input_source):
            print(f"Searching for ENVI images in directory: '{input_source}'")
            envi_files = glob.glob(os.path.join(input_source, '*.hdr'))
        elif os.path.isfile(input_source) and input_source.lower().endswith('.hdr'):
            envi_files = [input_source]
        else:
            print(f"Error: Input path '{input_source}' is not a valid file or directory.")
            return
    else:
        print(f"Error: 'input_source' must be a directory path (string) or a list of file paths. Got: {type(input_source)}")
        return

    if not envi_files:
        print("No valid ENVI header (.hdr) files found to process.")
        return

    print(f"\nFound {len(envi_files)} ENVI images to process.")

    # --- 3. Loop through each file and process it ---
    for hdr_path in envi_files:
        base_name = os.path.basename(hdr_path)
        file_name, file_extension = os.path.splitext(base_name)
        output_base_name = f"{file_name}_ref{file_extension}"
        output_hdr_path = os.path.join(output_dir, output_base_name)
        convert_to_reflectance(hdr_path, output_hdr_path, gain_path, offset_path)

    print("\n" + "="*80)
    print("--- Batch conversion complete. ---")
    print("="*80)

In [ ]:
INPUT_DIRECTORY = 'data/Greenhead_aug/temp'
OUTPUT_DIRECTORY = 'output/temp6'
INPUT_FILE_PATH = 'data/morven_9-2025/raw_36286_or.hdr'
GAIN_PATH = "gain.npy"
OFFSET_PATH = "offset.npy"

# Run the main function
batch_convert_to_reflectance(INPUT_FILE_PATH, OUTPUT_DIRECTORY, GAIN_PATH, OFFSET_PATH)